In [85]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, recall_score
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

In [86]:
# 데이터 불러오기
df = pd.read_csv('C:\\Users\\ansck\\Desktop\\Project\\2st_project\\churn_cleaned.csv')
df.head()

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,0,1,42,2,0.00,1,1,1,101348.88,1
1,608,1,1,41,1,83807.86,1,0,1,112542.58,0
2,502,0,1,42,8,159660.80,3,1,0,113931.57,1
3,699,0,1,39,1,0.00,2,0,0,93826.63,0
4,850,1,1,43,2,125510.82,1,1,1,79084.10,0


In [87]:
# 데이터 분할
X = df.drop('churn', axis=1)
y = df['churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [88]:
# 컬럼 분리
num_cols = ['credit_score', 'age', 'balance', 'estimated_salary', 'tenure']
cat_cols = ['country', 'gender', 'credit_card', 'active_member', 'products_number']

In [89]:
# 스케일러 및 인코더 정의
standard_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()
encoder = OneHotEncoder(drop='first', handle_unknown='ignore') 

# baseline model 선정
- 총 8개의 아무런 파라미터도 넣지 않은 기본 모델들을 학습
- 과적합 여부를 판단하기 위해 train, test 데이터 모두 평가 진행
- 차이가 가장 적은 RandomForest, AdaBoost, LogisticRegression 선정

In [ ]:
# 모델 정의
models = {
    'LogisticRegression': {
        'scaler': standard_scaler,
        'model': LogisticRegression()
    },
    'KNN': {
        'scaler': minmax_scaler,
        'model': KNeighborsClassifier()
    },
    'SVM': {
        'scaler': standard_scaler,
        'model': SVC()
    },
    'DecisionTree': {
        'scaler': None,
        'model': DecisionTreeClassifier()
    },
    'RandomForest': {
        'scaler': None,
        'model': RandomForestClassifier()
    },
    'Bagging': {
        'scaler': None,
        'model': BaggingClassifier()
    },
    'AdaBoost': {
        'scaler': None,
        'model': AdaBoostClassifier()
    },
    'NN': {
        'scaler': minmax_scaler,
        'model': MLPClassifier()
    }
}

In [91]:
# 결과 저장용 df
base_test_results = pd.DataFrame(columns=['Model', 'ROC_AUC', 'Accuracy', 'F1', 'Recall'])

In [92]:
# 학습 루프
for name, cfg in models.items():
    print(f"\n🧠 모델: {name}")
    # 전처리기 구성
    numeric_transformer = cfg['scaler'] if cfg['scaler'] else 'passthrough'
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_cols),
            ('cat', encoder, cat_cols)
        ]
    )
    
    # 파이프라인 구성
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', cfg['model'])
    ])
    
    # 예측 및 평가
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1] if hasattr(pipe.named_steps['model'], "predict_proba") else None
    
    # classification report 출력
    print(classification_report(y_test, y_pred))

    roc = roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan
    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    
    base_test_results.loc[len(base_test_results)] = [name, roc, acc, f1, rec]


🧠 모델: LogisticRegression
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1536
           1       0.75      0.46      0.57       390

    accuracy                           0.86      1926
   macro avg       0.81      0.71      0.74      1926
weighted avg       0.85      0.86      0.85      1926


🧠 모델: KNN
              precision    recall  f1-score   support

           0       0.87      0.95      0.90      1536
           1       0.67      0.42      0.52       390

    accuracy                           0.84      1926
   macro avg       0.77      0.68      0.71      1926
weighted avg       0.83      0.84      0.83      1926


🧠 모델: SVM
              precision    recall  f1-score   support

           0       0.87      0.98      0.92      1536
           1       0.83      0.41      0.55       390

    accuracy                           0.86      1926
   macro avg       0.85      0.70      0.73      1926
weighted avg       0.86   

c:\Users\ansck\miniconda3\envs\2st_project\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [112]:
# 모델 평가
base_test_results = base_test_results.sort_values(by='F1', ascending=False).reset_index(drop=True)
print("\n📊 Base_test 모델별 성능 비교 결과")
display(base_test_results)


📊 Base_test 모델별 성능 비교 결과


,Model,ROC_AUC,Accuracy,F1,Recall
0,AdaBoost,0.846413,0.863448,0.602118,0.510256
1,NN,0.853923,0.860852,0.588957,0.492308
2,RandomForest,0.850462,0.863448,0.581876,0.469231
3,Bagging,0.821947,0.857736,0.577160,0.479487
4,LogisticRegression,0.848725,0.859813,0.571429,0.461538
5,SVM,0.827879,0.863448,0.550427,0.412821
6,KNN,0.799461,0.840602,0.516535,0.420513
7,DecisionTree,0.693565,0.799585,0.510152,0.515385


In [94]:
# 결과 저장용 df - train 데이터
base_train_results = pd.DataFrame(columns=['Model', 'ROC_AUC', 'Accuracy', 'F1', 'Recall'])

In [95]:
# 학습 루프
for name, cfg in models.items():
    print(f"\n🧠 모델: {name}")
    # 전처리기 구성
    numeric_transformer = cfg['scaler'] if cfg['scaler'] else 'passthrough'
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_cols),
            ('cat', encoder, cat_cols)
        ]
    )
    
    # 파이프라인 구성
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', cfg['model'])
    ])
    
    # 예측 및 평가
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_train)
    y_proba = pipe.predict_proba(X_train)[:, 1] if hasattr(pipe.named_steps['model'], "predict_proba") else None
    
    # classification report 출력
    print(classification_report(y_train, y_pred))

    roc = roc_auc_score(y_train, y_proba) if y_proba is not None else np.nan
    acc = accuracy_score(y_train, y_pred)
    f1  = f1_score(y_train, y_pred)
    rec = recall_score(y_train, y_pred)
    
    base_train_results.loc[len(base_train_results)] = [name, roc, acc, f1, rec]


🧠 모델: LogisticRegression
              precision    recall  f1-score   support

           0       0.87      0.96      0.91      6141
           1       0.72      0.43      0.54      1559

    accuracy                           0.85      7700
   macro avg       0.79      0.69      0.72      7700
weighted avg       0.84      0.85      0.84      7700


🧠 모델: KNN
              precision    recall  f1-score   support

           0       0.89      0.96      0.92      6141
           1       0.79      0.52      0.62      1559

    accuracy                           0.87      7700
   macro avg       0.84      0.74      0.77      7700
weighted avg       0.87      0.87      0.86      7700


🧠 모델: SVM
              precision    recall  f1-score   support

           0       0.87      0.98      0.92      6141
           1       0.85      0.43      0.57      1559

    accuracy                           0.87      7700
   macro avg       0.86      0.70      0.75      7700
weighted avg       0.87   

c:\Users\ansck\miniconda3\envs\2st_project\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [109]:
# 모델 평가
base_train_results = base_train_results.sort_values(by='F1', ascending=False).reset_index(drop=True)
print("\n📊 Base_train 모델별 성능 비교 결과")
display(base_train_results)


📊 Base_train 모델별 성능 비교 결과


,Model,ROC_AUC,Accuracy,F1,Recall
0,DecisionTree,1.000000,1.000000,1.000000,1.000000
1,RandomForest,1.000000,1.000000,1.000000,1.000000
2,Bagging,0.999456,0.986234,0.964970,0.936498
3,KNN,0.923098,0.873506,0.622773,0.515715
4,NN,0.880269,0.866883,0.608031,0.509942
5,SVM,0.868368,0.868312,0.567775,0.427197
6,AdaBoost,0.847989,0.850000,0.566929,0.484926
7,LogisticRegression,0.847785,0.850390,0.538831,0.431687


# 점수 올리기
- 타켓 변수의 클랙스 불균형을 맞추기 위해 class_weight='balanced'로 설정
- 위의 3개 모델을 하이퍼 파라메터 튜닝을 통해 최적의 파라메터 값을 찾고 학습 진행
- 이또한 과적합이 일어났는지 확인하기 위해 train, test 데이터 모두 평가 진행

In [97]:
# 모델 정의
models = {
    'LogisticRegression': {
        'scaler': standard_scaler,
        'model': LogisticRegression(class_weight='balanced') 
    },
    'RandomForest': {
        'scaler': None,
        'model': RandomForestClassifier(class_weight='balanced')
    },
    'AdaBoost': {
        'scaler': None,
        'model': AdaBoostClassifier()
    }
}

In [98]:
# 결과 저장용 df
RSCV_test_results = pd.DataFrame(columns=['Model', 'ROC_AUC', 'Accuracy', 'F1', 'Recall'])

In [99]:
# 범위 기반 RandomizedSearchCV를 위한 라이브러리 추가
from scipy.stats import uniform, randint

best_params_dict = {}

# 학습 루프
for name, cfg in models.items():
    print(f"\n🧠 모델: {name}")
    # 전처리기 구성
    numeric_transformer = cfg['scaler'] if cfg['scaler'] else 'passthrough'
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_cols),
            ('cat', encoder, cat_cols)
        ]
    )
    
    # 파이프라인 구성
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', cfg['model'])
    ])

    # 범위 기반 RandomizedSearchCV 설정
    param_distributions = {}
    
    if name == 'LogisticRegression':
        param_distributions = {
            'model__C': uniform(0.001, 99.999),  # 0.001~100 사이 연속값
            'model__penalty': ['l1', 'l2'],
            'model__solver': ['liblinear', 'saga'],
            'model__max_iter': randint(500, 2001)  # 500~2000 사이 정수
        }
    elif name == 'RandomForest':
        param_distributions = {
            'model__n_estimators': randint(50, 501),  # 50~500 사이 정수
            'model__max_depth': randint(3, 21),       # 3~20 사이 정수 (None은 별도 처리)
            'model__min_samples_split': randint(2, 21),  # 2~20 사이 정수
            'model__min_samples_leaf': randint(1, 11),   # 1~10 사이 정수
            'model__max_features': ['sqrt', 'log2']      # 범주형 선택
        }
    elif name == 'AdaBoost':
        param_distributions = {
            'model__n_estimators': randint(50, 301),     # 50~300 사이 정수
            'model__learning_rate': uniform(0.01, 1.99), # 0.01~2.0 사이 연속값
        }
    
    # 범위 기반 RandomizedSearchCV 적용
    random_search = RandomizedSearchCV(
        pipe,
        param_distributions=param_distributions,
        n_iter=30,  # 30번의 랜덤 조합 시도
        cv=5,       # 5-fold 교차검증
        scoring='f1',  # F1-score 최적화
        n_jobs=-1,  # 모든 CPU 코어 사용
        random_state=42,
        verbose=1   # 진행상황 출력
    )
    
    # 예측 및 평가
    random_search.fit(X_train, y_train)
    best_pipe = random_search.best_estimator_

    # 최적 하이퍼파라미터 저장
    best_params_dict[name] = random_search.best_params_

    y_pred = best_pipe.predict(X_test)
    y_proba = best_pipe.predict_proba(X_test)[:, 1] if hasattr(best_pipe.named_steps['model'], "predict_proba") else None
    
    # classification report 출력
    print(classification_report(y_test, y_pred))

    roc = roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan
    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    
    RSCV_test_results.loc[len(RSCV_test_results)] = [name, roc, acc, f1, rec]


🧠 모델: LogisticRegression
Fitting 5 folds for each of 30 candidates, totalling 150 fits
              precision    recall  f1-score   support

           0       0.93      0.79      0.85      1536
           1       0.48      0.75      0.58       390

    accuracy                           0.78      1926
   macro avg       0.70      0.77      0.72      1926
weighted avg       0.84      0.78      0.80      1926


🧠 모델: RandomForest
Fitting 5 folds for each of 30 candidates, totalling 150 fits
              precision    recall  f1-score   support

           0       0.92      0.89      0.90      1536
           1       0.61      0.69      0.64       390

    accuracy                           0.85      1926
   macro avg       0.76      0.79      0.77      1926
weighted avg       0.85      0.85      0.85      1926


🧠 모델: AdaBoost
Fitting 5 folds for each of 30 candidates, totalling 150 fits
              precision    recall  f1-score   support

           0       0.88      0.96      0.92

In [100]:
# 최적의 파라메터 값 출력
display(best_params_dict)

{'LogisticRegression': {'model__C': np.float64(77.96932033627665),
  'model__max_iter': 1544,
  'model__penalty': 'l1',
  'model__solver': 'saga'},
 'RandomForest': {'model__max_depth': 10,
  'model__max_features': 'sqrt',
  'model__min_samples_leaf': 2,
  'model__min_samples_split': 16,
  'model__n_estimators': 102},
 'AdaBoost': {'model__learning_rate': np.float64(1.227189789371679),
  'model__n_estimators': 285}}

In [110]:
# 모델 평가
RSCV_test_results = RSCV_test_results.sort_values(by='F1', ascending=False).reset_index(drop=True)
print("\n📊 After Tuning_test 모델별 성능 비교 결과")
display(RSCV_test_results)


📊 After Tuning_test 모델별 성능 비교 결과


,Model,ROC_AUC,Accuracy,F1,Recall
0,RandomForest,0.859689,0.846314,0.644231,0.687179
1,AdaBoost,0.846388,0.864486,0.592824,0.487179
2,LogisticRegression,0.847294,0.784008,0.584830,0.751282


In [102]:
# 모델 정의
models = {
    'LogisticRegression': {
        'scaler': standard_scaler,
        'model': LogisticRegression(class_weight='balanced') 
    },
    'RandomForest': {
        'scaler': None,
        'model': RandomForestClassifier(class_weight='balanced')
    },
    'AdaBoost': {
        'scaler': None,
        'model': AdaBoostClassifier()
    }
}

In [103]:
# 결과 저장용 df
RSCV_train_results = pd.DataFrame(columns=['Model', 'ROC_AUC', 'Accuracy', 'F1', 'Recall'])

In [104]:
# 범위 기반 RandomizedSearchCV를 위한 라이브러리 추가
from scipy.stats import uniform, randint

best_params_dict = {}

# 학습 루프
for name, cfg in models.items():
    print(f"\n🧠 모델: {name}")
    # 전처리기 구성
    numeric_transformer = cfg['scaler'] if cfg['scaler'] else 'passthrough'
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_cols),
            ('cat', encoder, cat_cols)
        ]
    )
    
    # 파이프라인 구성
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', cfg['model'])
    ])

    # 범위 기반 RandomizedSearchCV 설정
    param_distributions = {}
    
    if name == 'LogisticRegression':
        param_distributions = {
            'model__C': uniform(0.001, 99.999),  # 0.001~100 사이 연속값
            'model__penalty': ['l1', 'l2'],
            'model__solver': ['liblinear', 'saga'],
            'model__max_iter': randint(500, 2001)  # 500~2000 사이 정수
        }
    elif name == 'RandomForest':
        param_distributions = {
            'model__n_estimators': randint(50, 501),  # 50~500 사이 정수
            'model__max_depth': randint(3, 21),       # 3~20 사이 정수 (None은 별도 처리)
            'model__min_samples_split': randint(2, 21),  # 2~20 사이 정수
            'model__min_samples_leaf': randint(1, 11),   # 1~10 사이 정수
            'model__max_features': ['sqrt', 'log2']      # 범주형 선택
        }
    elif name == 'AdaBoost':
        param_distributions = {
            'model__n_estimators': randint(50, 301),     # 50~300 사이 정수
            'model__learning_rate': uniform(0.01, 1.99), # 0.01~2.0 사이 연속값
        }
    
    # 범위 기반 RandomizedSearchCV 적용
    random_search = RandomizedSearchCV(
        pipe,
        param_distributions=param_distributions,
        n_iter=30,  # 30번의 랜덤 조합 시도
        cv=5,       # 5-fold 교차검증
        scoring='f1',  # F1-score 최적화
        n_jobs=-1,  # 모든 CPU 코어 사용
        random_state=42,
        verbose=1   # 진행상황 출력
    )
    
    # 예측 및 평가
    random_search.fit(X_train, y_train)
    best_pipe = random_search.best_estimator_

    # 최적 하이퍼파라미터 저장
    best_params_dict[name] = random_search.best_params_

    y_pred = best_pipe.predict(X_train)
    y_proba = best_pipe.predict_proba(X_train)[:, 1] if hasattr(best_pipe.named_steps['model'], "predict_proba") else None
    
    # classification report 출력
    print(classification_report(y_train, y_pred))

    roc = roc_auc_score(y_train, y_proba) if y_proba is not None else np.nan
    acc = accuracy_score(y_train, y_pred)
    f1  = f1_score(y_train, y_pred)
    rec = recall_score(y_train, y_pred)
    
    RSCV_train_results.loc[len(RSCV_train_results)] = [name, roc, acc, f1, rec]


🧠 모델: LogisticRegression
Fitting 5 folds for each of 30 candidates, totalling 150 fits
              precision    recall  f1-score   support

           0       0.93      0.78      0.85      6141
           1       0.47      0.76      0.58      1559

    accuracy                           0.77      7700
   macro avg       0.70      0.77      0.71      7700
weighted avg       0.83      0.77      0.79      7700


🧠 모델: RandomForest
Fitting 5 folds for each of 30 candidates, totalling 150 fits
              precision    recall  f1-score   support

           0       0.96      0.90      0.93      6141
           1       0.68      0.83      0.75      1559

    accuracy                           0.89      7700
   macro avg       0.82      0.87      0.84      7700
weighted avg       0.90      0.89      0.89      7700


🧠 모델: AdaBoost
Fitting 5 folds for each of 30 candidates, totalling 150 fits
              precision    recall  f1-score   support

           0       0.88      0.95      0.91

In [105]:
# 최적의 파라메터 값 출력
display(best_params_dict)

{'LogisticRegression': {'model__C': np.float64(37.454637344617396),
  'model__max_iter': 1360,
  'model__penalty': 'l1',
  'model__solver': 'liblinear'},
 'RandomForest': {'model__max_depth': 10,
  'model__max_features': 'sqrt',
  'model__min_samples_leaf': 3,
  'model__min_samples_split': 2,
  'model__n_estimators': 428},
 'AdaBoost': {'model__learning_rate': np.float64(1.227189789371679),
  'model__n_estimators': 285}}

In [111]:
# 모델 평가 - train 데이터
RSCV_train_results = RSCV_train_results.sort_values(by='F1', ascending=False).reset_index(drop=True)
print("\n📊 After Tuning_train 모델별 성능 비교 결과")
display(RSCV_train_results)


📊 After Tuning_train 모델별 성능 비교 결과


,Model,ROC_AUC,Accuracy,F1,Recall
0,RandomForest,0.953425,0.886883,0.748919,0.833226
1,AdaBoost,0.857212,0.857662,0.577487,0.480436
2,LogisticRegression,0.848041,0.774675,0.576313,0.756895


# 모델을 .pkl 파일로 저장

In [ ]:
# 모델 저장을 위한 라이브러리 추가
import joblib
import pickle
from datetime import datetime

# 모델 평가 후 상위 모델들 저장
RSCV_test_results = RSCV_test_results.sort_values(by='F1', ascending=False).reset_index(drop=True)
print("\n📊 모델별 성능 비교 결과")
display(RSCV_test_results)

# 성능 top3 모델 출력
top3_models = RSCV_test_results.head(3)
print("\n🏆 상위 3개 모델:")
display(top3_models)

# 📦 모델 저장 섹션
print("\n💾 모델 저장 시작...")

# 저장할 모델들 다시 학습 (최적 파라미터로)
saved_models = {}
model_performances = {}

# 3개 모델 모두 저장하도록 수정
for name, cfg in models.items():
    if name in ['RandomForest', 'AdaBoost', 'LogisticRegression']:  # 3개 모델 모두 저장
        print(f"\n🔄 {name} 모델 재학습 및 저장...")
        
        # 전처리기 구성
        numeric_transformer = cfg['scaler'] if cfg['scaler'] else 'passthrough'
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, num_cols),
                ('cat', encoder, cat_cols)
            ]
        )
        
        # 파이프라인 구성
        pipe = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', cfg['model'])
        ])

        # 하이퍼파라미터 설정
        param_distributions = {}
        
        if name == 'LogisticRegression':
            param_distributions = {
                'model__C': uniform(0.001, 99.999),
                'model__penalty': ['l1', 'l2'],
                'model__solver': ['liblinear', 'saga'],
                'model__max_iter': randint(500, 2001)
            }
        elif name == 'RandomForest':
            param_distributions = {
                'model__n_estimators': randint(50, 501),
                'model__max_depth': randint(3, 21),
                'model__min_samples_split': randint(2, 21),
                'model__min_samples_leaf': randint(1, 11),
                'model__max_features': ['sqrt', 'log2']
            }
        elif name == 'AdaBoost':
            param_distributions = {
                'model__n_estimators': randint(50, 301),
                'model__learning_rate': uniform(0.01, 1.99)
            }
        
        # RandomizedSearchCV로 최적 모델 찾기
        random_search = RandomizedSearchCV(
            pipe,
            param_distributions=param_distributions,
            n_iter=30,
            cv=5,
            scoring='f1',
            n_jobs=-1,
            random_state=42,
            verbose=0  # 저장 시에는 verbose 끄기
        )
        
        # 학습
        print(f"🔍 {name} 하이퍼파라미터 튜닝 중...")
        random_search.fit(X_train, y_train)
        best_model = random_search.best_estimator_
        
        # 테스트 성능 평가
        y_pred = best_model.predict(X_test)
        y_proba = best_model.predict_proba(X_test)[:, 1]
        
        f1 = f1_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)
        
        # 성능 저장
        model_performances[name] = {
            'F1': f1,
            'Recall': recall,
            'Accuracy': accuracy,
            'ROC_AUC': roc_auc,
            'Best_Params': random_search.best_params_,
            'Best_CV_Score': random_search.best_score_
        }
        
        # 모델 저장
        saved_models[name] = best_model
        
        print(f"✅ {name} 성능: F1={f1:.3f}, Recall={recall:.3f}, ROC_AUC={roc_auc:.3f}")
        print(f"🎯 최적 파라미터: {random_search.best_params_}")

# 📁 파일로 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

print(f"\n💾 모델 파일 저장 중... (타임스탬프: {timestamp})")

for model_name, model in saved_models.items():
    # joblib로 저장 (권장)
    joblib_filename = f'{model_name.lower()}_model_{timestamp}.pkl'
    joblib.dump(model, joblib_filename)
    
    # pickle로도 저장 (백업)
    pickle_filename = f'{model_name.lower()}_model_{timestamp}.pickle'
    with open(pickle_filename, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"💾 {model_name} 저장 완료:")
    print(f"   - Joblib: {joblib_filename}")
    print(f"   - Pickle: {pickle_filename}")

# 📊 성능 정보도 저장
performance_filename = f'model_performances_{timestamp}.pkl'
joblib.dump(model_performances, performance_filename)
print(f"\n📈 성능 정보 저장: {performance_filename}")

# 📋 전체 요약 정보 저장
summary_info = {
    'saved_date': timestamp,
    'models_saved': list(saved_models.keys()),
    'performance_summary': model_performances,
    'data_info': {
        'train_size': len(X_train),
        'test_size': len(X_test),
        'num_features': num_cols,
        'cat_features': cat_cols,
        'total_features': len(num_cols) + len(cat_cols)
    },
    'model_rankings': RSCV_test_results.to_dict('records')
}

summary_filename = f'model_summary_{timestamp}.pkl'
joblib.dump(summary_info, summary_filename)
print(f"📋 모델 요약 정보 저장: {summary_filename}")

print(f"\n🎉 모델 저장 완료! 총 {len(saved_models)}개 모델 저장됨")
print(f"📂 저장된 파일들:")
print(f"   • 모델 파일: {len(saved_models)*2}개 (.pkl, .pickle)")
print(f"   • 성능 정보: {performance_filename}")
print(f"   • 요약 정보: {summary_filename}")